# Sensitivity analysis for the DFN

Example showing how to perform sensitivity analysis for the DFN with PyBaMM

In [1]:
%pip install pybamm -q
import pybamm
import numpy as np

Note: you may need to restart the kernel to use updated packages.


Load model

In [2]:
model = pybamm.lithium_ion.DFN()

Before performing a sensitivity analysis, we scale the parameters with their reference values. Some parameters are functions of states, but we can evaluate them at appropriate values of the states to obtain a reference value. In this notebook, we choose to study the effect on the voltage of:
- negative particle diffusivity (via Ds_n)
- positive particle diffusivity (via Ds_p)
- electrolyte diffusivity (via D_e)
- electrolyte conductivity (via kappa_e)
- negative electrode kinetics (via j0_n)
- positive electrode kinetics (via j0_p)

In [62]:
ce_ref = param["Typical electrolyte concentration [mol.m-3]"]
csn_ref = param["Maximum concentration in negative electrode [mol.m-3]"]
T_ref = param["Reference temperature [K]"]
param.evaluate(param["Negative electrode exchange-current density [A.m-2]"](ce_ref, 0.5 * csn_ref, T_ref))

7.900401128126567

In [122]:
param = model.default_parameter_values
# Get reference values for evaluating functions
c_e_ref = param["Typical electrolyte concentration [mol.m-3]"]
csn_ref = param["Maximum concentration in negative electrode [mol.m-3]"]
csp_ref = param["Maximum concentration in positive electrode [mol.m-3]"]
T_ref = param["Reference temperature [K]"]
# Evaluate functions at reference values
Dsn_ref = param["Negative electrode diffusivity [m2.s-1]"](0.5, T_ref).evaluate()
Dsp_ref = param["Positive electrode diffusivity [m2.s-1]"](0.5, T_ref).evaluate()
De_ref = param["Electrolyte diffusivity [m2.s-1]"](c_e_ref, T_ref).evaluate()
kappae_ref = param["Electrolyte conductivity [S.m-1]"](c_e_ref, T_ref).evaluate()
j0n_ref = param.evaluate(param["Negative electrode exchange-current density [A.m-2]"](ce_ref, 0.5 * csn_ref, T_ref))
j0p_ref = param.evaluate(param["Positive electrode exchange-current density [A.m-2]"](ce_ref, 0.5 * csp_ref, T_ref))

In [123]:
param["Negative electrode diffusivity [m2.s-1]"] = Dsn_ref * pybamm.InputParameter("Dsn")
# param["Positive electrode diffusivity [m2.s-1]"] = Dsp_ref * pybamm.InputParameter("Dsp")
# param["Electrolyte diffusivity [m2.s-1]"] = De_ref * pybamm.InputParameter("D_e")
# param["Electrolyte conductivity [S.m-1]"] = kappae_ref * pybamm.InputParameter("kappa_e")
# param["Negative electrode exchange-current density [A.m-2]"] = j0n_ref * pybamm.InputParameter("j0n")
# param["Positive electrode exchange-current density [A.m-2]"] = j0p_ref * pybamm.InputParameter("j0p")

Create simulation, run and read solution

In [124]:
solver = pybamm.CasadiSolver(extra_options_setup={"ad_weight": 0})
sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
solution = sim.solve(t_eval=np.linspace(0,3600,5))

Since we have not specified the parameter values when solving, the resulting solution contains _symbolic_ variables, such as the voltage

In [125]:
V = solution["Terminal voltage [V]"]
V

Now we can evaluate the voltage at specific values for the input parameters to get both the value

In [128]:
%%time
V.value({"Dsn": 1, "Dsp": 1, "D_e": 1, "kappa_e": 1, "j0n": 1, "j0p": 1})

CPU times: user 144 ms, sys: 2.01 ms, total: 146 ms
Wall time: 146 ms


DM([[3.77171, 3.67082, 3.61315, 3.58406, 3.16798]])

and sensitivity

In [129]:
%%time
sens = V.sensitivity({"Dsn": 1, "Dsp": 1, "D_e": 1, "kappa_e": 1, "j0n": 1, "j0p": 1})

CPU times: user 672 ms, sys: 7.61 ms, total: 679 ms
Wall time: 675 ms


In [109]:
sens

DM(
[[00, 00], 
 [0.00144585, 0.00372077], 
 [0.00652415, 0.00146917], 
 [0.00246126, 0.00137044], 
 [0.0174035, 0.223563]])

In [84]:
inputs = {k: 1 for k in V.symbolic_inputs_dict.keys()}
h = 1e-6
for k in inputs.keys():
    V_down = V.value(inputs)
    inputs[k] = 1 + h
    V_up = V.value(inputs)
    sens = (V_up - V_down) / h
    print(sens)
    inputs[k] = 1

[[0, 0.000422464, 0.000444994, 0.000428074, 0.000407487, 0.000399877, 0.000428391, 0.000542233, 0.000778531, 0.00103886, 0.00122177, 0.0013259, 0.00137903, 0.0014072, 0.00143188, 0.00147267, 0.00155152, 0.00170014, 0.00197684, 0.002514, 0.00366832, 0.00623007, 0.00951514, 0.0100351, 0.00790636, 0.00546389, 0.00381094, 0.00281181, 0.0021213, 0.00151904, 0.00102273, 0.00104159, 0.00145708, 0.00182547, 0.00207236, 0.00222589, 0.00231668, 0.0023674, 0.00239664, 0.00242538, 0.00249296, 0.00270719, 0.00340515, 0.00526309, 0.0081467, 0.0109712, 0.0132931, 0.015046, 0.0162617, 0.0170824]]
[[0.00485232, 0.00523816, 0.0052792, 0.0052831, 0.00527848, 0.00525259, 0.00517244, 0.00495439, 0.00451041, 0.00397247, 0.00352681, 0.00321071, 0.00300636, 0.00289322, 0.0028568, 0.00288818, 0.00298289, 0.00314032, 0.0033631, 0.00365425, 0.00399468, 0.00423495, 0.00415848, 0.0040753, 0.00417632, 0.00439649, 0.00459593, 0.0047219, 0.00481469, 0.00494027, 0.00510006, 0.00503174, 0.00469119, 0.00435765, 0.004107

In [82]:
inputs

{'Dsn': 1,
 'j0n': 1,
 'Dsp': 1,
 'j0p': 1,
 'D_e': 1,
 'kappa_e': 1,
 0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1}